# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import re
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet'])
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier
from sklearn.neural_network import MLPClassifier
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,HashingVectorizer,TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,accuracy_score
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\savad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\savad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///cleanDatabase.db')
df = pd.read_sql_table('cleanData',engine)
df['related']=df['related'].replace(2,0)
X = df[['message']].values
Y = df.loc[:,'related':'direct_report'].values

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    regex_express='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(regex_express, text)
    for url in detected_urls:
        text =text.replace(url,'url_link')
        
    regex_express='^[a-zA-Z0-9]'
    detected_spc = re.findall(regex_express, text)
    for spc in detected_spc:
        text =text.replace(spc,' ')
        
    tokens = nltk.tokenize.word_tokenize(text)
    lemmatizer =  nltk.stem.WordNetLemmatizer() 
    
    clean_tokens = []
    for tok in tokens:
        clean_tok =lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BaggingClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.15, random_state=42)

In [5]:
pipeline.fit(X.ravel(), Y)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None,
         verbose=0, warm_start=False),
           n_jobs=None))])

In [6]:
y_pred=pipeline.predict(X_test.ravel())

In [7]:
ypred=pd.DataFrame(y_pred,columns=df.loc[:,'related':'direct_report'].columns)
yreal=pd.DataFrame(Y_test,columns=df.loc[:,'related':'direct_report'].columns)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
for coll in ypred.columns:
    print('-----'+coll+'-----')
    print(classification_report(yreal[coll], ypred[coll]))
    print('Accuracy score : {}'.format(accuracy_score(yreal[coll], ypred[coll])))

-----related-----
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       972
           1       0.98      1.00      0.99      2960

   micro avg       0.98      0.98      0.98      3932
   macro avg       0.99      0.97      0.98      3932
weighted avg       0.98      0.98      0.98      3932

Accuracy score : 0.9847405900305188
-----request-----
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      3283
           1       0.99      0.85      0.91       649

   micro avg       0.97      0.97      0.97      3932
   macro avg       0.98      0.92      0.95      3932
weighted avg       0.97      0.97      0.97      3932

Accuracy score : 0.9735503560528993
-----offer-----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3910
           1       1.00      0.64      0.78        22

   micro avg       1.00      1.00      1.00      3932
   m

Accuracy score : 0.9974567650050865
-----shops-----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3918
           1       1.00      0.86      0.92        14

   micro avg       1.00      1.00      1.00      3932
   macro avg       1.00      0.93      0.96      3932
weighted avg       1.00      1.00      1.00      3932

Accuracy score : 0.9994913530010173
-----aid_centers-----
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3885
           1       1.00      0.74      0.85        47

   micro avg       1.00      1.00      1.00      3932
   macro avg       1.00      0.87      0.93      3932
weighted avg       1.00      1.00      1.00      3932

Accuracy score : 0.9969481180061037
-----other_infrastructure-----
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      3744
           1       1.00      0.72      0.84       188

   mi

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'clf__estimator__max_features':[2,6,10,100]
}
cv = GridSearchCV(pipeline, parameters).fit(X_train.ravel(), Y_train)
print(cv.best_score_)    
print(cv.best_params_)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.19613916947250282
{'clf__estimator__max_features': 2}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BaggingClassifier(max_features=2)))
    ])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.15, random_state=42)
pipeline.fit(X.ravel(), Y)
y_pred=pipeline.predict(X_test.ravel())
ypred=pd.DataFrame(y_pred,columns=df.loc[:,'related':'direct_report'].columns)
yreal=pd.DataFrame(Y_test,columns=df.loc[:,'related':'direct_report'].columns)
for coll in ypred.columns:
    print('-----'+coll+'-----')
    print(classification_report(yreal[coll], ypred[coll]))
    print('Accuracy score : {}'.format(accuracy_score(yreal[coll], ypred[coll])))

-----related-----
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       972
           1       0.75      1.00      0.86      2960

   micro avg       0.75      0.75      0.75      3932
   macro avg       0.38      0.50      0.43      3932
weighted avg       0.57      0.75      0.65      3932

Accuracy score : 0.7527975584944049
-----request-----
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      3283
           1       0.00      0.00      0.00       649

   micro avg       0.83      0.83      0.83      3932
   macro avg       0.42      0.50      0.46      3932
weighted avg       0.70      0.83      0.76      3932

Accuracy score : 0.8349440488301119
-----offer-----
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3910
           1       0.00      0.00      0.00        22

   micro avg       0.99      0.99      0.99      3932
   m

Accuracy score : 0.9476093591047813
-----direct_report-----
              precision    recall  f1-score   support

           0       0.81      1.00      0.89      3177
           1       0.00      0.00      0.00       755

   micro avg       0.81      0.81      0.81      3932
   macro avg       0.40      0.50      0.45      3932
weighted avg       0.65      0.81      0.72      3932

Accuracy score : 0.8079857578840285


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
features=[TfidfVectorizer(tokenizer=tokenize)]
parameters = {
    'clf__estimator__learning_rate':[0.1,0.5,1]
}

for faet in features:
    pipeline = Pipeline([
    ('vect', faet),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    cv = GridSearchCV(pipeline, parameters).fit(X_train.ravel(), Y_train)
    print(cv.best_score_)    
    print(cv.best_params_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.19703703703703704
{'clf__estimator__learning_rate': 0.1}


### 9. Export your model as a pickle file

In [12]:
pickle.dump(pipeline, open('model_to_use.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.